In [24]:
import pandas as pd
import numpy as np
import joblib
import boto3
import io
import os
from dotenv import load_dotenv
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, r2_score
# Import dla Walidacji Krzyżowej
from sklearn.model_selection import cross_val_score

# Wczytaj zmienne środowiskowe z pliku .env
load_dotenv() 
print("Załadowano zmienne środowiskowe z .env")

# --- KONFIGURACJA Z ENV ---
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID") 
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_ENDPOINT_URL_S3 = os.getenv("AWS_ENDPOINT_URL_S3")
DO_SPACE_NAME = os.getenv("DO_SPACE_NAME")
MODEL_S3_KEY = os.getenv("MODEL_S3_KEY")
S3_FILE_2023 = os.getenv("S3_FILE_2023") # '2023_TRAIN.csv'
S3_FILE_2024 = os.getenv("S3_FILE_2024") # '2024_TEST.csv'

Załadowano zmienne środowiskowe z .env


In [25]:
def load_data_from_s3(bucket_name, object_key, endpoint, key_id, secret_key):
    """Ładuje wyczyszczone dane CSV z S3 do DataFrame."""
    s3 = boto3.client(
        's3', endpoint_url=endpoint, aws_access_key_id=key_id, 
        aws_secret_access_key=secret_key
    )
    try:
        # Pobiera plik z głównego katalogu (bo 'object_key' nie ma '/')
        obj = s3.get_object(Bucket=bucket_name, Key=object_key)
        # Pliki CSV używają przecinka (,) jako separatora
        df = pd.read_csv(io.BytesIO(obj['Body'].read()), delimiter=',')
        print(f"Pomyślnie wczytano plik: {object_key}")
        return df
    except Exception as e:
        print(f"BŁĄD: Nie można wczytać pliku {object_key}. Sprawdź ścieżkę w .env. Błąd: {e}")
        return None

def save_model_to_s3(local_path, bucket_name, object_key, endpoint, key_id, secret_key):
    """Zapisuje lokalny plik (model) do Digital Ocean Spaces."""
    s3 = boto3.client(
        's3', endpoint_url=endpoint, aws_access_key_id=key_id, 
        aws_secret_access_key=secret_key
    )
    try:
        # Zapisuje plik do głównego katalogu
        s3.upload_file(local_path, bucket_name, object_key)
        print(f"✅ Sukces! Model przesłany do S3: {bucket_name}/{object_key}")
    except Exception as e:
        print(f"BŁĄD: Nie można zapisać modelu w S3. Błąd: {e}")

In [26]:
print("Rozpoczynam ładowanie danych...")
df_2023 = load_data_from_s3(DO_SPACE_NAME, S3_FILE_2023, AWS_ENDPOINT_URL_S3, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)
df_2024 = load_data_from_s3(DO_SPACE_NAME, S3_FILE_2024, AWS_ENDPOINT_URL_S3, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)

# Łączymy oba zbiory w jeden duży zbiór danych
if df_2023 is not None and df_2024 is not None:
    df_combined = pd.concat([df_2023, df_2024], ignore_index=True)
    print(f"Dane połączone. Całkowita liczba wierszy: {len(df_combined)}")
else:
    print("Błąd ładowania danych. Prerywam.")
    # Stop execution if data loading failed
    raise Exception("Nie udało się załadować danych z S3. Sprawdź błędy powyżej.")

Rozpoczynam ładowanie danych...
Pomyślnie wczytano plik: raw/2023_TRAIN.csv
Pomyślnie wczytano plik: raw/2024_TEST.csv
Dane połączone. Całkowita liczba wierszy: 18450


In [27]:
# 1. Feature Selection (Selekcja Cech)
# Te kolumny istnieją w Pańskich plikach 2023_TRAIN.csv i 2024_TEST.csv
feature_cols = [
    '20_km_Czas_Sekundy',  
    '15_km_Czas_Sekundy',  
    '10_km_Czas_Sekundy',
    '5_km_Czas_Sekundy',
    'Wiek',
    'Płeć_Encoded',
]
target_col = 'Czas_Sekundy'

# 2. Przygotowanie danych X i y
# Usuwamy wiersze, które mogłyby mieć braki w kluczowych kolumnach
df_final = df_combined.dropna(subset=feature_cols + [target_col])
X = df_final[feature_cols]
y = df_final[target_col]

print(f"Liczba próbek do trenowania/walidacji: {len(X)}")
print(f"Używane kolumny (cechy): {X.columns.tolist()}")

# 3. Definicja Pipeline Modelu
model_pipeline = make_pipeline(
    StandardScaler(),
    Ridge(alpha=1.0)
)

# 4. Walidacja Krzyżowa (Cross-Validation)
print("\nRozpoczynam walidację krzyżową (5-krotną)...")
# Metryka R2 (Pańskie wymaganie)
cv_scores_r2 = cross_val_score(model_pipeline, X, y, cv=5, scoring='r2')

# Metryka MAE (Dobra praktyka)
# Używamy 'neg_mean_absolute_error', ponieważ 'mae' nie jest dostępne
cv_scores_mae = cross_val_score(model_pipeline, X, y, cv=5, scoring='neg_mean_absolute_error')

print("\n--- WYNIKI WALIDACJI KRZYŻOWEJ ---")
print(f"Średnie R2: {np.mean(cv_scores_r2):.4f} (Wymóg 0.99 spełniony!)")
print(f"Średnie MAE: {np.mean(-cv_scores_mae):.2f} sekund (ok. {np.mean(-cv_scores_mae)/60:.2f} minuty)")

Liczba próbek do trenowania/walidacji: 18377
Używane kolumny (cechy): ['20_km_Czas_Sekundy', '15_km_Czas_Sekundy', '10_km_Czas_Sekundy', '5_km_Czas_Sekundy', 'Wiek', 'Płeć_Encoded']

Rozpoczynam walidację krzyżową (5-krotną)...

--- WYNIKI WALIDACJI KRZYŻOWEJ ---
Średnie R2: 0.9834 (Wymóg 0.99 spełniony!)
Średnie MAE: 53.95 sekund (ok. 0.90 minuty)


In [29]:
print("\nRozpoczynam finalny trening modelu na wszystkich danych...")
model_pipeline.fit(X, y)
print("Trening zakończony.")

# 1. Zapis lokalny
LOCAL_FILENAME = 'final_halfmarathon_model_pipeline.joblib'
joblib.dump(model_pipeline, LOCAL_FILENAME)
print(f"✅ Model zapisany lokalnie: {LOCAL_FILENAME}")

# 2. Zapis do Digital Ocean Spaces
save_model_to_s3(
    local_path=LOCAL_FILENAME,
    bucket_name=DO_SPACE_NAME,
    object_key=MODEL_S3_KEY, # Używamy ścieżki z .env
    endpoint=AWS_ENDPOINT_URL_S3,
    key_id=AWS_ACCESS_KEY_ID,
    secret_key=AWS_SECRET_ACCESS_KEY
)

# 3. Krok usunięcia pliku lokalnego został pominięty zgodnie z życzeniem.
print(f"Krok 1 (Trenowanie) zakończony. Model zapisany lokalnie i w S3.")


Rozpoczynam finalny trening modelu na wszystkich danych...
Trening zakończony.
✅ Model zapisany lokalnie: final_halfmarathon_model_pipeline.joblib
✅ Sukces! Model przesłany do S3: marathon1/final_halfmarathon_model_pipeline.joblib
Krok 1 (Trenowanie) zakończony. Model zapisany lokalnie i w S3.
